In [1]:
import warnings
import itertools

import pandas as pd
import numpy as np
from sktime.performance_metrics.forecasting import mean_squared_error
from sktime.forecasting.model_selection import (
    ForecastingRandomizedSearchCV,
    ExpandingWindowSplitter,
)
from sktime.utils.plotting import plot_series

from sklearn.ensemble import RandomForestRegressor
from sktime.forecasting.compose import make_reduction
from sktime.forecasting.stream import UpdateEvery

from src.data import load_train_test
from src import get_config, plot_windows, get_rolling_predictions

In [19]:
config = get_config()
train, test = load_train_test(config.processed_data_path)
train.tail()

,90,102,101,107,65,103,missing,precipitation,festivos
timestamps,,,,,,,,,
2022-11-14 19:00:00,5.0,2.0,5.0,6.0,5.0,14.0,0,0.0,0
2022-11-14 20:00:00,10.0,2.0,4.0,6.0,6.0,13.0,0,0.0,0
2022-11-14 21:00:00,7.0,12.0,4.0,5.0,4.0,9.0,0,0.0,0
2022-11-14 22:00:00,9.0,7.0,4.0,5.0,7.0,12.0,0,0.0,0
2022-11-14 23:00:00,8.0,7.0,4.0,5.0,7.0,10.0,0,0.0,0


In [20]:
def shift_columns(dataframe, columns, k):
    for column in columns:
        new_column_name = f"{column}_hace_{k}_horas"
        dataframe[new_column_name] = dataframe[column].shift(k)
    return dataframe


exo_stations = ["102", "101", "107", "65", "103"]
train = shift_columns(train, exo_stations, 1)
test = shift_columns(test, exo_stations, 1)
# Fix test first row
test.iloc[0, 9:] = train.iloc[-1, 1:6].values
test.head()

,90,102,101,107,65,103,missing,precipitation,festivos,102_hace_1_horas,101_hace_1_horas,107_hace_1_horas,65_hace_1_horas,103_hace_1_horas
timestamps,,,,,,,,,,,,,,
2022-11-15 00:00:00,5.0,7.0,3.0,4.0,6.0,10.0,0,0.0,0,7.0,4.0,5.0,7.0,10.0
2022-11-15 01:00:00,4.0,7.0,3.0,4.0,7.0,9.0,0,0.0,0,7.0,3.0,4.0,6.0,10.0
2022-11-15 02:00:00,5.0,7.0,3.0,2.0,8.0,8.0,0,0.0,0,7.0,3.0,4.0,7.0,9.0
2022-11-15 03:00:00,5.0,6.0,3.0,3.0,8.0,8.0,0,0.0,0,7.0,3.0,2.0,8.0,8.0
2022-11-15 04:00:00,5.0,6.0,3.0,3.0,8.0,8.0,0,0.1,0,6.0,3.0,3.0,8.0,8.0


In [ ]:
# Add lagged target a week ago
train = shift_columns(train, [str(config.target_station_id)], 24 * 7)

In [ ]:
# Filter train data to use oly data since september 2022
train = train[train.index >= "2022-09-01"]

target_column = str(config.target_station_id)
train_target_series = train[target_column]
test_target_series = test[target_column]

In [ ]:
random_forest = make_reduction(
    RandomForestRegressor(n_estimators=200, random_state=42),
    window_length=24 * 7,
    strategy="direct",
)